In [4]:
!pip install TensorFlowTTS

  Using cached TensorFlowTTS-1.8-py3-none-any.whl.metadata (24 kB)
INFO: pip is looking at multiple versions of tensorflowtts to determine which version is compatible with other requirements. This could take a while.
  Using cached TensorFlowTTS-1.6.1-py3-none-any.whl.metadata (23 kB)
  Using cached TensorFlowTTS-1.6-py3-none-any.whl.metadata (23 kB)
  Using cached TensorFlowTTS-1.1-py3-none-any.whl.metadata (22 kB)
  Using cached TensorFlowTTS-0.11-py3-none-any.whl.metadata (22 kB)
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [44 lines of output]
  Traceback (most recent call last):
    File "C:\Users\vinit\anaconda3\Lib\site-packages\setuptools\_vendor\packaging\requirements.py", line 35, in __init__
      parsed = _parse_requirement(requirement_string)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\vinit\anaconda3\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 64, in parse_requirement
      return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\vinit\anaconda3\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 82, in _parse_requirement
      url, specifier, marker = _parse_requirement_details(tokenizer)
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\vinit\anaconda3\Lib\site-packages\setuptools\_v

In [2]:
# imports
import tensorflow as tf
import numpy as np
import pandas as pd
from tacotron2_model import create_tacotron2_model
from tensorflow_tts.optimizers import AdamWeightDecay
from tensorflow_tts.losses import TacotronLoss
from tensorflow_tts.utils import Char2MelProcessor
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tensorflow_tts'

In [ ]:
# loading the data
# metadata.csv has two transcriptions for each audio file - split the csv by '|'.
metadata = pd.read_csv("metadata.csv", sep='|', names=["ID", "transcription1", "transcription2"])

# creating a new column for audio paths 
# metadata.csv contains 'ID's, which are the names of the audio files in the wavs dir
# for example, the ID LJ001-0001 correlates to LJ001-0001.wav in the wavs folder, so we append
# .wav to the end of the ID and get the audio
metadata['audio_path'] = metadata['ID'].apply(lambda x: f'wavs/{x}.wav')

# using train_test_split to generate training vs testing data
train_metadata, val_metadata = train_test_split(metadata, test_size=0.1, random_state=42)


In [ ]:
# hyperparameters
BATCH_SIZE = 8
EPOCHS = 50

In [ ]:
# initializing a tacotron2 model
tacotron2 = create_tacotron2_model()

# defining our optimizer - read on the Adam optimizer from class 
optimizer = AdamWeightDecay(learning_rate=1e-4, weight_decay_rate=1e-6)

# compiling the model
tacotron2.compile(optimizer=optimizer, loss=TacotronLoss())


In [ ]:
# training the model
history = tacotron2.fit(train_metadata["transcription1"], train_metadata["audio_path"],
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=(val_metadata["transcription1"], val_metadata["audio_path"]))
